In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
print('gerando datas')
datas = pd.date_range(start = '2011-01-01', end = pd.to_datetime('today'), freq = 'M')

d = {}
str_datas = [str(x) for x in datas]

for dt in str_datas:    
    header = {'ACCEPT':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
    baseURL = f'https://www.npr.org/series/423302056/hidden-brain/archive?date={dt}'
    req = requests.request('GET', baseURL, headers = header, data = {})

    soup = BeautifulSoup(req.text)

    for x in soup.find_all():
            if f'npr.org/{dt[:4]}/' in str(x.get('href')) and len(x.get_text()) > 3 :
                link = x.get('href')
                text = x.get_text()
                if link not in list(d):
                    d[link] = [text]    
                else:
                    if text not in d[link]:
                        d[link].append(x.get_text())

    

gerando datas


ParserError: Unknown string format: June 10, 2011  Anthony Weiner, Arnold Schwarzenegger — men behaving badly, right? It may be more complex than that. New survey research shows that, given power, women are as likely as men to stray.

In [47]:
import pandas as pd 
import re 
df = pd.DataFrame(d).T
df = df.reset_index()
df.columns  = ['link','title','description']
df['date'] = pd.to_datetime(df.description.apply(lambda x: re.findall('[A-Za-z0-9\s,]*[\d]{4}',x)[0]))

df['description'] = df.description.apply(lambda x: re.findall('\d{4}..(.*)',x)[0])
df = df.sort_values('date', ascending = False)
df


,link,title,description,date
604,https://www.npr.org/2020/09/25/916997530/laugh...,Laughter: The Best Medicine,"If you listen closely to giggles, guffaws, an...",2020-09-28
605,https://www.npr.org/2020/08/21/904660038/the-h...,The Halo Effect,"Judy, Lyn and Donna Ulrich were driving to a ...",2020-09-21
606,https://www.npr.org/2020/09/14/912749547/why-n...,Why Nobody Feels Rich,If you've ever flown in economy class on a pl...,2020-09-14
607,https://www.npr.org/2020/09/02/908728981/slayi...,The Fee-for-Service Monster,The United States spends trillions of dollars...,2020-09-07
598,https://www.npr.org/2020/08/31/907943965/you-2...,You 2.0: Empathy Gym,Some people are good at putting themselves in...,2020-08-31
...,...,...,...,...
3,https://www.npr.org/2011/08/31/140085048/to-do...,"To Dodge Blame, Officials Prep Public For Worst",Because people are more likely to blame publi...,2011-08-31
4,https://www.npr.org/2011/08/02/138922339/under...,"Under Pressure, Soccer Goalies Tend To Dive Right",Here's some science for the betting table: Wh...,2011-08-02
1,https://www.npr.org/2011/07/07/137660609/its-a...,Dispute: How Violent Video Games Affect Kids,Scientists have long clashed over whether vio...,2011-07-07
2,https://www.npr.org/2011/07/04/137526401/the-k...,"The Key To Disaster Survival? Pals, Neighbors","A researcher's data suggest that ambulances, ...",2011-07-04
